### Question 1

#### Part 1

In [88]:
import arviz as az
import numpy as np
import pymc as pm
from pymc.math import switch, ge
import pandas as pd

In [ ]:
data = pd.read_csv("PONV.csv")
X = data[["Gender", "Anaesthesiaduration", "Smoking", "PONVhist"]].to_numpy()
X_aug = np.concatenate((np.ones((X.shape[0], 1)), X), axis=1)
y = data["SinclairScore"].to_numpy()

In [90]:
with pm.Model() as m:
    # Associate data with model
    X_data = pm.Data("X", X_aug)
    y_data = pm.Data("y", y)

    # Priors
    beta = pm.Normal("beta", mu=0, sigma=1000, shape=X.shape[1] + 1)
    tau = pm.Gamma("tau", alpha=0.001, beta=0.001)  # Precision
    sigma = pm.Deterministic("sigma", 1 / pm.math.sqrt(tau))

    mu = pm.math.dot(X_data, beta)

    # Likelihood
    pm.Normal("likelihood", mu=mu, sigma=sigma, observed=y_data)

    # Start sampling
    trace = pm.sample(5000, target_accept=0.95)
    ppc = pm.sample_posterior_predictive(trace, extend_inferencedata=True)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, tau]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 44 seconds.
Sampling: [likelihood]


Output()

In [91]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
beta[0],-0.067,0.010,-0.086,-0.046,0.000,0.000,13223.0,12740.0,1.0
beta[1],0.096,0.007,0.084,0.109,0.000,0.000,16509.0,13586.0,1.0
beta[2],0.002,0.000,0.002,0.002,0.000,0.000,16566.0,14256.0,1.0
beta[3],0.004,0.005,-0.006,0.013,0.000,0.000,19529.0,14316.0,1.0
beta[4],-0.030,0.005,-0.040,-0.019,0.000,0.000,19701.0,12957.0,1.0
tau,225.635,10.475,205.672,246.481,0.076,0.054,19021.0,12953.0,1.0
sigma,0.067,0.002,0.064,0.070,0.000,0.000,19021.0,12953.0,1.0


#### Part 2

In [92]:
# Code is based on Unit 6.8 Prediction
# Single prediction on out-of-sample data
new_ob = np.array([[1, 1, 55, 0, 1]])
pm.set_data({"X": new_ob}, model=m)
ppc_new = pm.sample_posterior_predictive(trace, model=m, predictions=True)

Sampling: [likelihood]


Output()

In [93]:
az.summary(ppc_new.predictions, hdi_prob=0.95).mean()

mean             0.119989
sd               0.066918
hdi_2.5%        -0.010727
hdi_97.5%        0.250978
mcse_mean        0.000000
mcse_sd          0.000000
ess_bulk     19819.669214
ess_tail     19577.520742
r_hat            1.000000
dtype: float64

#### Part 3

In [94]:
y_pred = ppc.posterior_predictive.stack(sample=("chain", "draw"))["likelihood"].values.T
az.r2_score(y, y_pred)

r2        0.485581
r2_std    0.014191
dtype: float64

### Question 2

#### Part 1

In [95]:
# Create dictionary with "Lemon yellow" = 1, "White" = 2, "Green" = 3, and "Blue" = 4
data = {
    1: [45, 59, 48, 46, 38, 47],
    2: [21, 12, 14, 17, 13, 17],
    3: [16, 11, 20, 21, 14, 7],
    4: [37, 32, 15, 25, 39, 41],
}

In [96]:
# The following code is adapted based on Aaron Reding's OH on 10/30/24
with pm.Model() as m:
    mu0 = pm.Normal("mu0", mu=0, sigma=10)
    sigma = pm.InverseGamma("sigma", 0.1, 0.1)

    alpha4 = pm.Normal("alpha4", mu=0, sigma=10)
    alpha3 = pm.Normal("alpha3", mu=0, sigma=10)
    alpha2 = pm.Normal("alpha2", mu=0, sigma=10)
    # Sum-to-zero constraint
    alpha1 = pm.Deterministic("alpha1", -(alpha2 + alpha3 + alpha4))

    mu1 = mu0 + alpha1
    mu2 = mu0 + alpha2
    mu3 = mu0 + alpha3
    mu4 = mu0 + alpha4

    pm.Normal("likelihood1", mu=mu1, sigma=sigma, observed=data[1])
    pm.Normal("likelihood2", mu=mu2, sigma=sigma, observed=data[2])
    pm.Normal("likelihood3", mu=mu3, sigma=sigma, observed=data[3])
    pm.Normal("likelihood4", mu=mu4, sigma=sigma, observed=data[4])

    onetwo = pm.Deterministic("alpha1 - alpha2", alpha1 - alpha2)
    onethree = pm.Deterministic("alpha1 - alpha3", alpha1 - alpha3)
    onefour = pm.Deterministic("alpha1 - alpha4", alpha1 - alpha4)
    twothree = pm.Deterministic("alpha2 - alpha3", alpha2 - alpha3)
    twofour = pm.Deterministic("alpha2 - alpha4", alpha2 - alpha4)
    threefour = pm.Deterministic("alpha3 - alpha4", alpha3 - alpha4)

    trace = pm.sample(5000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [mu0, sigma, alpha4, alpha3, alpha2]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 27 seconds.


In [97]:
az.summary(trace, hdi_prob=0.95)

,mean,sd,hdi_2.5%,hdi_97.5%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
mu0,26.717,1.461,23.834,29.560,0.011,0.008,18496.0,12810.0,1.0
alpha4,3.477,2.434,-1.389,8.336,0.017,0.013,19572.0,13765.0,1.0
alpha3,-11.866,2.409,-16.519,-6.946,0.018,0.013,18066.0,14154.0,1.0
alpha2,-11.090,2.422,-15.743,-6.140,0.018,0.012,19037.0,13575.0,1.0
sigma,7.049,1.171,5.016,9.439,0.010,0.007,13887.0,13287.0,1.0
alpha1,19.478,2.493,14.324,24.131,0.015,0.011,26682.0,15925.0,1.0
alpha1 - alpha2,30.568,4.055,22.646,38.652,0.027,0.019,22673.0,17468.0,1.0
alpha1 - alpha3,31.344,4.007,23.271,39.173,0.027,0.019,22438.0,16398.0,1.0
alpha1 - alpha4,16.001,4.022,7.893,23.852,0.026,0.018,24783.0,17313.0,1.0
alpha2 - alpha3,0.776,3.902,-7.177,8.200,0.029,0.029,17764.0,13466.0,1.0


#### Part 2
Please see the accompanying report for the conclusions based on the output above.

### Question 3

#### Part 1

In [98]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.utils import resample

#### Frequentist Logistic Regression Method #1

In [99]:
data_df = pd.read_csv("iris-1.csv")
data_df = data_df.rename(
    columns={
        "Sepal.Length": "Sepal_Length",
        "Sepal.Width": "Sepal_Width",
        "Petal.Length": "Petal_Length",
        "Petal.Width": "Petal_Width",
    }
)

In [100]:
# Convert Species into 0 and 1
le = LabelEncoder()
data_df["Species"] = le.fit_transform(data_df["Species"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'setosa': 0, 'versicolor': 1}

In [101]:
data_df.head()

,Sepal_Length,Sepal_Width,Petal_Length,Petal_Width,Species
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [102]:
x = data_df.iloc[:, :4]
formula = "Species ~ " + "+".join(x.columns)
logit_v1 = smf.logit(formula=formula, data=data_df).fit()
logit_v1.summary()

         Current function value: 0.000000
         Iterations: 35


c:\Users\jm121\Bayes\Lib\site-packages\statsmodels\discrete\discrete_model.py:227: PerfectSeparationWarning: Perfect separation or prediction detected, parameter may not be identified
  warnings.warn(msg, category=PerfectSeparationWarning)
c:\Users\jm121\Bayes\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Species   No. Observations:                  100
Model:                          Logit   Df Residuals:                       95
Method:                           MLE   Df Model:                            4
Date:                Sun, 03 Nov 2024   Pseudo R-squ.:                   1.000
Time:                        15:27:36   Log-Likelihood:            -8.9814e-06
converged:                      False   LL-Null:                       -69.315
Covariance Type:            nonrobust   LLR p-value:                 5.547e-29
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        9.6813    1.2e+04      0.001      0.999   -2.35e+04    2.35e+04
Sepal_Length    -4.1173   3316.583     -0.001      0.999   -6504.500    6496.265
Sepal_Width     -8.9814   1815.027     -0.005      0.996   -3566.370    3548.407
Petal_Length     4.4103   1631.257      0.003      0.998   -3192.795    3201.615
Petal_Width     33.8138   5245.781      0.006      0.995   -1.02e+04    1.03e+04
================================================================================

Complete Separation: The results show that there iscomplete separation or perfect prediction.
In this case the Maximum Likelihood Estimator does not exist and the parameters
are not identified.
"""

#### Frequentist Logistic Regression Method #2 w/ L1 Regularization

In [103]:
# L1 Lasso regularization
logit_v2 = smf.logit(formula=formula, data=data_df).fit_regularized()

logit_v2.summary()

Optimization terminated successfully    (Exit mode 0)
            Current function value: 9.602598429097631e-11
            Iterations: 34
            Function evaluations: 35
            Gradient evaluations: 34


<class 'statsmodels.iolib.summary.Summary'>
"""
                           Logit Regression Results                           
==============================================================================
Dep. Variable:                Species   No. Observations:                  100
Model:                          Logit   Df Residuals:                       95
Method:                           MLE   Df Model:                            4
Date:                Sun, 03 Nov 2024   Pseudo R-squ.:                   1.000
Time:                        15:27:36   Log-Likelihood:            -9.6026e-09
converged:                       True   LL-Null:                       -69.315
Covariance Type:            nonrobust   LLR p-value:                 5.547e-29
================================================================================
                   coef    std err          z      P>|z|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       -1.9979   9.39e+06  -2.13e-07      1.000   -1.84e+07    1.84e+07
Sepal_Length    -8.2904   3.78e+06  -2.19e-06      1.000   -7.42e+06    7.42e+06
Sepal_Width    -14.3042   7.15e+06     -2e-06      1.000    -1.4e+07     1.4e+07
Petal_Length    28.5257   2.95e+06   9.68e-06      1.000   -5.77e+06    5.77e+06
Petal_Width     11.7497   7.26e+06   1.62e-06      1.000   -1.42e+07    1.42e+07
================================================================================

Complete Separation: The results show that there iscomplete separation or perfect prediction.
In this case the Maximum Likelihood Estimator does not exist and the parameters
are not identified.
"""

#### Frequentist Logistic Regression Method #3 w/ L2 Regularization

In [104]:
# L2 Ridge regularization
x = data_df.iloc[:, :4]
intercept_arr = np.ones(len(x))
x.insert(0, "Intercept", pd.Series(intercept_arr))
y = data_df["Species"]


def fit_model(x, y):
    model = LogisticRegression(penalty="l2", random_state=42)
    model.fit(x, y)
    return model.coef_[0]


# Number of bootstrap iterations
n_iterations = 100
n_samples = x.shape[0]

bootstrap_coefficients = []

for i in range(n_iterations):
    indices = resample(range(n_samples), replace=True, n_samples=n_samples)
    sample_x = x.iloc[indices, :]
    sample_y = y[indices]

    # Fit model and append coefficients
    coefficients = fit_model(sample_x, sample_y)
    bootstrap_coefficients.append(coefficients)

# Convert to numpy array
bootstrap_coefficients = np.array(bootstrap_coefficients)

# Calculate confidence intervals
conf_int = np.percentile(bootstrap_coefficients, [2.5, 97.5], axis=0)

# Fit model for point estimates
model = LogisticRegression(penalty="l2", fit_intercept=False, random_state=42)
model.fit(x, y)
point_estimates = model.coef_[0]

summary = pd.DataFrame(
    {
        "Feature": x.columns,
        "Coefficient": point_estimates,
        "Lower CI": conf_int[0],
        "Upper CI": conf_int[1],
    }
)

In [105]:
summary

,Feature,Coefficient,Lower CI,Upper CI
0,Intercept,-0.258227,-0.006872,0.005881
1,Sepal_Length,-0.402121,0.323307,0.640020
2,Sepal_Width,-1.464355,-1.052238,-0.660454
3,Petal_Length,2.237124,2.238369,2.367531
4,Petal_Width,1.000677,0.859751,1.057332


#### Part 2

#### Bayesian Logistic Regression Method w/ Uninformative Prior

In [106]:
data_df = pd.read_csv("iris-1.csv")

In [107]:
# Convert Species into 0 and 1
le = LabelEncoder()
data_df["Species"] = le.fit_transform(data_df["Species"])
le_name_mapping = dict(zip(le.classes_, le.transform(le.classes_)))
le_name_mapping

{'setosa': 0, 'versicolor': 1}

In [108]:
x = data_df.iloc[:, 0:4]
y = data_df["Species"]

y.head()

0    0
1    0
2    0
3    0
4    0
Name: Species, dtype: int32

In [109]:
with pm.Model() as logistic:
    x_data = pm.Data("x_data", x)
    y_data = pm.Data("y_data", y)

    alpha = pm.Normal("alpha", mu=0, sigma=np.sqrt(1000))
    betas = pm.Normal("betas", mu=0, sigma=np.sqrt(1000), shape=x.shape[1])

    p = alpha + pm.math.dot(x_data, betas)

    pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(5000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, betas]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 77 seconds.
There were 4976 divergences after tuning. Increase `target_accept` or reparameterize.


In [110]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-5.570,29.798,-60.818,51.004,0.548,0.387,2964.0,4412.0,1.0
betas[0],-6.570,15.832,-38.030,21.366,0.362,0.256,1908.0,2802.0,1.0
betas[1],-21.991,21.062,-63.394,15.485,0.446,0.315,2246.0,3452.0,1.0
betas[2],35.810,20.220,-0.246,73.895,0.436,0.308,2109.0,3116.0,1.0
betas[3],18.091,27.191,-32.647,69.528,0.479,0.339,3222.0,4123.0,1.0


#### Part 3

#### Bayesian Logistic Regression Method w/ Informative Prior

In [111]:
with pm.Model() as logistic:
    x_data = pm.Data("x_data", x)
    y_data = pm.Data("y_data", y)

    alpha = pm.Normal("alpha", mu=0, sigma=1)
    betas = pm.Normal("betas", mu=0, sigma=1, shape=x.shape[1])

    p = alpha + pm.math.dot(x_data, betas)

    pm.Bernoulli("y", logit_p=p, observed=y_data)

    trace = pm.sample(5000)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [alpha, betas]


Output()

Sampling 4 chains for 1_000 tune and 5_000 draw iterations (4_000 + 20_000 draws total) took 38 seconds.
There were 3 divergences after tuning. Increase `target_accept` or reparameterize.


In [112]:
az.summary(trace)

,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_bulk,ess_tail,r_hat
alpha,-0.294,0.974,-2.159,1.500,0.009,0.007,11877.0,11262.0,1.0
betas[0],-0.422,0.583,-1.532,0.675,0.007,0.005,7736.0,8530.0,1.0
betas[1],-1.590,0.738,-2.975,-0.188,0.008,0.006,8834.0,10540.0,1.0
betas[2],2.428,0.616,1.273,3.573,0.007,0.005,8744.0,10218.0,1.0
betas[3],1.080,0.926,-0.656,2.876,0.009,0.006,11164.0,10853.0,1.0
